<a href="https://colab.research.google.com/github/Bread806/goldbach_backup_from_colab/blob/main/goldbach_0709_EKim_experiment_modprime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**粗體文字** <br>
# E kim mod 25的實驗
<br>



In [4]:
from google.colab import drive
drive.mount("/content/Drive")

Mounted at /content/Drive


In [2]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LayerNormalization
from random import choice, sample


def is_prime(num):
    """檢查一個數字是否為質數"""
    if num <= 1:
        return False
    sqr_num = int(num ** 0.5)
    # for i in range(2, sqr_num+ 1):
    for i in range(2, sqr_num+ 1):
        if num % i == 0:
            return False
    return True


def prime_table(x):
    """建立小於x的質數表"""
    primes = [num for num in range(2, x) if is_prime(num)]
    return primes


def convert_base_into_list(number, base, width=10):
    result = []  # init list

    for i in range(width):
        result.append(number % base)
        number = number // base
    result = result[::-1]
    return result


def prime_to_index(primeSize, primes, number):
    for index in range(primeSize):
        if number == primes[index]:
            return index
    return -1

def add_prime(first25Prime, n):
    result = []
    if n < 100:
      for item in first25Prime:
        if item > n :break
        else : result.append(n%item)
    else :
      for item in first25Prime:
        result.append(n%item)

    return [0]*(25-len(result)) + result

def prime_base(n, p, length=10):
  digit = []
  if n==0:
    return '0'.zfill(length)
  while n and len(digit)<10:
    digit.append(int(n%p))
    n = n//p
  return [0]*(10 - len(digit)) + digit


In [ ]:
n = 1212
p = 3

prime_base(n,p)

[0, 0, 0, 0, 2, 2, 2, 2, 1, 1]

In [5]:
# shuffle

print("---loading data---")
# load data
trainData = pd.read_csv('/content/Drive/MyDrive/實驗/goldbach/csv/traindata_min_size_1000.csv')
trainDataNumbers = trainData['Number'].values
trainDataLabel = trainData['Factors'].values
print("---loading data done.---")

# shuffle training data
shuffleIndices = np.random.permutation(len(trainDataNumbers))
shuffledNumbers = trainDataNumbers[shuffleIndices]
shuffledLabel = trainDataLabel[shuffleIndices]

---loading data---
---loading data done.---


In [ ]:
print (shuffledNumbers[5])
print (shuffledLabel [5])

3887790
34182


In [6]:
# loadding prime table
primeTable = pd.read_csv("/content/Drive/MyDrive/實驗/goldbach/csv/prime_table_5000000.csv")
primeTableNumber = primeTable["primes"].values
print ("---loading primes table---")

---loading primes table---


In [8]:
first25Prime = primeTableNumber[:25]

In [13]:
# collect data
# prepare training set
mergedTrainNumber = []
mergedTrainLabel = []
flag = 1
for num in shuffledNumbers:

  # Euler's Totient Function Partitions fist 50
  mergedTrainNumber.append(add_prime(first25Prime, num))

for num in shuffledLabel:
    mergedTrainLabel.append(num)

#print (mergedTrainNumber)

In [14]:
mergedArray = np.array(mergedTrainNumber)
mergedLabel = np.array(mergedTrainLabel)

In [15]:
print (mergedArray)

[[ 0  1  3 ...  4 64 77]
 [ 0  1  4 ... 45 29 64]
 [ 0  0  2 ... 44 14  4]
 ...
 [ 0  0  2 ... 33 32 60]
 [ 0  0  3 ... 69 17 14]
 [ 0  2  4 ... 34 32 58]]


In [16]:
X_train, X_val, y_train, y_val = train_test_split(mergedArray, mergedLabel, test_size=0.2, random_state=42)

In [17]:
print (X_train[0].shape)

(25,)


In [33]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as keras

# 清空所有
keras.clear_session()

# 設置早停回調
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 模型訓練設置
device = "/device:GPU:0"
with tf.device(device):
    print(f"--- gpu {device} is loaded. ---")
    model = Sequential()
    model.add(Dense(units=256, input_shape=(25,), activation='relu'))
    model.add(Dense(units=256, activation='relu'))
    model.add(Dense(units=256, activation='relu'))
    model.add(Dense(units=256, activation='relu'))
    model.add(Dense(units=256, activation='relu'))
    model.add(Dense(units=1, activation='linear'))

    print(model.summary())

    custom_adam_optimizer = Adam(learning_rate=0.0002)
    model.compile(loss='MSE', optimizer=custom_adam_optimizer, metrics=['accuracy'])

    # 開始訓練
    print("---starting training---")
    trainHistory = model.fit(X_train, y_train, epochs=200, batch_size=128, validation_data=(X_val, y_val), callbacks=[early_stopping])
    print("--------------------------------------------------------")

    # 保存模型
    model.save('./csv/goldbach_model_0709_kim.h5')
print("model saved.")


--- gpu /device:GPU:0 is loaded. ---
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               6656      
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 256)               65792     
                                                                 
 dense_4 (Dense)             (None, 256)               65792     
                                                                 
 dense_5 (Dense)             (None, 1)                 257       
                                                                 
Total params: 27008

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## predict

In [22]:
from google.colab import drive
drive.mount("/content/Drive")

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


In [34]:
# loadding prime table
primeTable = pd.read_csv("/content/Drive/MyDrive/實驗/goldbach/csv/prime_table_5000000.csv")
primeTableNumber = primeTable["primes"].values
print ("---loading primes table---")

---loading primes table---


In [35]:
## load data
#predict

from tensorflow.keras.models import load_model
import csv
import pandas as pd
import numpy as np

# 指定 HDF5 模型文件的路徑
modelPath = '/content/csv/goldbach_model_0709_kim.h5'

# 使用 load_model
model = load_model(modelPath)

# load test_set and prime data
testDataPath = '/content/Drive/MyDrive/實驗/goldbach/csv/traindata_min_size_400000.csv'
testData = pd.read_csv(testDataPath)
testDataNumbers = testData['Number'].values
testDataLabel = testData['Factors'].values

print("---loading test data done.---")

---loading test data done.---


In [36]:
# shuffle training data
shuffleIndices = np.random.permutation(len(testDataNumbers))
shuffledNumbers = testDataNumbers[shuffleIndices]
shuffledLabel = testDataLabel[shuffleIndices]

In [37]:
# generate 25 prime table
primeTable25 = primeTableNumber[:25]
print (primeTable25)

[ 2  3  5  7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89
 97]


In [38]:
# collect data
# prepare training set
mergedTestNumber = []
mergedTestLabel = []
first25Prime = primeTableNumber[:25]

for num in shuffledNumbers:

   # Euler's Totient Function Partitions fist 50
    mergedTestNumber.append(add_prime(first25Prime, num))

for num in shuffledLabel:
    mergedTestLabel.append(num)

#print (mergedTrainNumber)

In [39]:
mergedArray = np.array(mergedTestNumber)
mergedLabel = np.array(mergedTestLabel)

In [40]:
evaluation = model.evaluate(mergedArray, mergedLabel)
XNewData = testDataNumbers, testDataLabel
print ("Loss:", evaluation[0])
print ("Accuracy:", evaluation[1])
print ("##########################")
predictions = model.predict(mergedArray)

12500/12500 [==============================] - 45s 4ms/step - loss: 54932144.0000 - accuracy: 5.0000e-06
Loss: 54932144.0
Accuracy: 4.999999873689376e-06
##########################
12500/12500 [==============================] - 43s 3ms/step


In [ ]:
for i in mergedLabel:
  print (i)


In [ ]:
print (predictions[0][0])
pickPredictions = []
for item in predictions:
  # pickPredictions.append(item[0])
  print(item[0])


In [ ]:
for i in pickPredictions:
  print (i)

In [41]:
# cal MSE, RMSE, error rate

import math

rmse = math.sqrt(evaluation[0])
testMean = (np.mean(testDataLabel))

print (f"MSE: {evaluation[0]}")
print (f"RMSE: {rmse}")
print (f"error rate: {round(rmse/testMean,3)} / ({round(rmse/testMean,3)*100}%)")

MSE: 54932144.0
RMSE: 7411.622224587543
error rate: 0.688 / (68.8%)


In [31]:
print (f"測試五十筆資料")
roundPredictions = np.round(predictions)
for index, number in enumerate(testDataNumbers):
  if index > 50 : break
  print (f"第{index}筆資料為數字:\t{number}\t|正確答案為:{testDataLabel[index]}\t模型預估為{predictions[index]}, 四捨五入到{roundPredictions[index]}")


測試五十筆資料
第0筆資料為數字:	2082910	|正確答案為:10145	模型預估為[11469.04], 四捨五入到[11469.]
第1筆資料為數字:	2218346	|正確答案為:8869	模型預估為[10710.174], 四捨五入到[10710.]
第2筆資料為數字:	2513286	|正確答案為:17760	模型預估為[9607.348], 四捨五入到[9607.]
第3筆資料為數字:	2636120	|正確答案為:12504	模型預估為[9448.209], 四捨五入到[9448.]
第4筆資料為數字:	1188388	|正確答案為:4752	模型預估為[10856.076], 四捨五入到[10856.]
第5筆資料為數字:	3471474	|正確答案為:24572	模型預估為[10862.633], 四捨五入到[10863.]
第6筆資料為數字:	917482	|正確答案為:3775	模型預估為[13140.481], 四捨五入到[13140.]
第7筆資料為數字:	1152574	|正確答案為:4611	模型預估為[6767.2393], 四捨五入到[6767.]
第8筆資料為數字:	2232150	|正確答案為:22406	模型預估為[11894.278], 四捨五入到[11894.]
第9筆資料為數字:	2527042	|正確答案為:10651	模型預估為[10274.844], 四捨五入到[10275.]
第10筆資料為數字:	3013418	|正確答案為:10603	模型預估為[10614.924], 四捨五入到[10615.]
第11筆資料為數字:	2556614	|正確答案為:9310	模型預估為[9494.686], 四捨五入到[9495.]
第12筆資料為數字:	962962	|正確答案為:5849	模型預估為[13213.182], 四捨五入到[13213.]
第13筆資料為數字:	441812	|正確答案為:2516	模型預估為[11659.505], 四捨五入到[11660.]
第14筆資料為數字:	1439516	|正確答案為:6749	模型預估為[8870.338], 四捨五入到[8870.]
第15筆資料為數字:	1913274	|正確答案為:15604	模型預估為[5642.857], 四捨五入到[5643.]
第

In [ ]:
print ("\t正確答案  "  , "\t/" ,"  \t模型預測")
for index, number in enumerate(testDataNumbers):
  print (f"{index}\t{testDataLabel[index]}  \t\t/  \t{roundPredictions[index]} = {testDataLabel[index]/roundPredictions[index]} ")
print ("###########################################################################")

print ("\t正確答案  "  , "\t-" ,"  \t模型預測")
for index, number in enumerate(testDataNumbers):
  print (f"{index}\t{testDataLabel[index]}  \t\t-  \t{roundPredictions[index]} = {testDataLabel[index]-roundPredictions[index]} ")

	正確答案   	/   	模型預測
0	16566  		/  	[7230.] = [2.2912862] 
1	31717  		/  	[7230.] = [4.3868604] 
2	14594  		/  	[7920.] = [1.8426768] 
3	14832  		/  	[7497.] = [1.9783914] 
4	15673  		/  	[7230.] = [2.1677732] 
5	16346  		/  	[8400.] = [1.9459524] 
6	13108  		/  	[16679.] = [0.78589845] 
7	18098  		/  	[7230.] = [2.5031812] 
8	31284  		/  	[13948.] = [2.2429023] 
9	42050  		/  	[13948.] = [3.014769] 
10	15623  		/  	[19052.] = [0.8200189] 
11	15886  		/  	[15414.] = [1.0306215] 
12	28364  		/  	[8657.] = [3.2764237] 
13	14047  		/  	[13948.] = [1.0070978] 
14	14997  		/  	[7230.] = [2.0742738] 
15	22961  		/  	[10404.] = [2.2069397] 
16	16070  		/  	[7230.] = [2.2226832] 
17	13302  		/  	[8968.] = [1.4832739] 
18	13996  		/  	[8657.] = [1.6167264] 
19	15871  		/  	[13948.] = [1.1378692] 
20	15845  		/  	[13948.] = [1.1360052] 
21	32717  		/  	[15249.] = [2.1455178] 
22	31204  		/  	[7343.] = [4.2494893] 
23	19410  		/  	[9523.] = [2.0382233] 
24	30918  		/  	[15263.] = [2.025683] 
25	232

In [ ]:
print ("模型預測差距")
for index, number in enumerate(testDataNumbers):
  print (int(testDataLabel[index]-roundPredictions[index]))

模型預測差距
9336
24487
6674
7335
8443
7946
-3571
10868
17336
28102
-3429
472
19707
99
7767
12557
8840
4334
5339
1923
1897
17468
23861
9887
15655
14404
8568
16593
9670
9807
18779
6672
9204
8689
5817
9991
8879
20869
-185
26750
5123
20440
5631
23044
19639
7127
-2355
11305
19897
12812


In [ ]:
print ("模型預測")
for index, number in enumerate(testDataNumbers):
  print (int(roundPredictions[index]))

模型預測
7230
7230
7920
7497
7230
8400
16679
7230
13948
13948
19052
15414
8657
13948
7230
10404
7230
8968
8657
13948
13948
15249
7343
9523
15263
8872
7230
14714
10545
7230
7711
7784
8657
8968
9523
8657
7493
13948
13948
7920
13948
9523
9523
7230
8657
7230
17332
7230
7230
8124
